# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта «Определение наиболее выгодого тарифа»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали.

Необходимо построить модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверить *accuracy* на тестовой выборке.

## Открывем и изучаем файл

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from sklearn.tree import DecisionTreeClassifier #дерево решений
from sklearn.ensemble import RandomForestClassifier #случайный лес
from sklearn.linear_model import LogisticRegression #логистическая регрессия


from sklearn.metrics import accuracy_score #функция для подсчёта accuracy
from sklearn.model_selection import train_test_split, GridSearchCV #разделение на выборки, подбор пвраметров для дерева решений


In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df= pd.read_csv('users_behavior.csv')

In [3]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


Датасет имеет 3214 объектов и 5 признаков

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 

Признаки:

* `сalls` — количество звонков
* `minutes` — суммарная длительность звонков в минутах
* `messages` — количество sms-сообщений
* `mb_used` — израсходованный интернет-трафик в Мб
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разбиваем данные на выборки

In [4]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

# так как мы не имеем отдельной тестовой выборки - разделяем эту на 3 в соотношении 60х20х20 %

#обучающая

features_train, features_x, target_train, target_x = train_test_split(
    features, target, test_size=0.40, random_state=12345)

# валидационная & тестовая 

features_test, features_valid, target_test, target_valid = train_test_split(
    features_x, target_x, test_size=0.50, random_state=12345)

display(target_train.shape) #обучающая
display(target_test.shape) #тестовая
display(target_valid.shape) #валидационная  

(1928,)

(643,)

(643,)

Обучающий набор данных сохранён в переменных features_train (признаки) и target_train (целевой признак).


## Исследуем модели

### Дерево решений

In [5]:
%%time
best_model = None
best_result = 0
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid) # предсказания модели
    result = accuracy_score(target_valid, predictions_valid) # качество модели
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        
print("accuracy лучшей модели:", best_result)
print("max_depth лучшей модели:", best_depth)

accuracy лучшей модели: 0.7993779160186625
max_depth лучшей модели: 7
CPU times: user 229 ms, sys: 0 ns, total: 229 ms
Wall time: 232 ms


Проверили какая длинна дерева даёт лучший результат accuray

In [6]:
%%time
#Осторожно!Исполняется около минуты 
model_tree = DecisionTreeClassifier()
model_tree.fit(features_train, target_train) 
parametry={
    'criterion':['gini', 'entropy'],
    'max_depth':range(1,10),
    'min_samples_split':range(1,10),
    'min_samples_leaf':range(1,5)
}
grid = GridSearchCV(model_tree,
                   param_grid=parametry,
                   cv=10,
                   verbose=1,
                   n_jobs=-1)
grid.fit(features_train, target_train)
display(grid.best_params_)
display(grid.best_score_)

Fitting 10 folds for each of 648 candidates, totalling 6480 fits


{'criterion': 'gini',
 'max_depth': 7,
 'min_samples_leaf': 1,
 'min_samples_split': 3}

0.8101845854922279

CPU times: user 1min 8s, sys: 55 ms, total: 1min 8s
Wall time: 1min 9s


С помощью функции GridSearchCV проверили какие гиперпараметры дадут наибольший accuray для всех значимых параметров

In [7]:
predictions_valid_tree = model.predict(features_valid)


### Cлучайный лес

In [8]:
%%time
best_model = None
best_result = 0
for est in range(1, 100):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_forest.fit(features_train, target_train) 
    predictions_valid_forest = model_forest.predict(features_valid) 
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model_forest
        best_result = result
        best_est = est
        
print("accuracy лучшей модели:", best_result)
print("n_estimators лучшей модели:", best_est)

accuracy лучшей модели: 0.7433903576982893
n_estimators лучшей модели: 1
CPU times: user 28.3 s, sys: 87.9 ms, total: 28.3 s
Wall time: 28.9 s


### Логистическая регрессия

In [9]:
%%time
model_regr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_regr.fit(features_train, target_train)
predictions_valid = model_regr.predict(features_valid)
accuracy = model_regr.score(features_valid, target_valid)
print(accuracy)

0.6842923794712286
CPU times: user 42.5 ms, sys: 3.99 ms, total: 46.5 ms
Wall time: 52.8 ms


In [10]:
%%time
model_regr = LogisticRegression(random_state=12345, penalty='l1', solver='saga', max_iter=1000)
model_regr.fit(features_train, target_train)
predictions_valid = model_regr.predict(features_valid)
accuracy= model_regr.score(features_valid, target_valid)
print(accuracy)

0.6842923794712286
CPU times: user 357 ms, sys: 3.95 ms, total: 361 ms
Wall time: 375 ms


In [11]:
%%time
model_regr = LogisticRegression(random_state=12345)
model_regr.fit(features_train, target_train)
predictions_valid_regr = model_regr.predict(features_valid)
accuracy= model_regr.score(features_valid, target_valid)
print(accuracy)

0.6842923794712286
CPU times: user 52.6 ms, sys: 32 µs, total: 52.6 ms
Wall time: 51.7 ms


## Проверяем модель на тестовой выборке

### Дерево решений

In [21]:
%%time
model_tree = DecisionTreeClassifier(random_state=12345, max_depth=7, min_samples_split=3)
model_tree.fit(features_train, target_train) 
predictions = model_tree.predict(features_test)
accuracy_tree = model_tree.score(features_test, target_test)
print(accuracy_tree)

0.7822706065318819
CPU times: user 14.1 ms, sys: 4.04 ms, total: 18.1 ms
Wall time: 15.4 ms


### Cлучайный лес

In [22]:
%%time
model_forest =  RandomForestClassifier(random_state=12345, n_estimators=48, max_depth=7, min_samples_split=3)
model_forest.fit(features_train, target_train) 
predictions = model_forest.predict(features_test)
accuracy_forest = model_forest.score(features_test, target_test)
print(accuracy_forest)

0.8055987558320373
CPU times: user 209 ms, sys: 4 µs, total: 209 ms
Wall time: 207 ms


### Логистическая регрессия

In [14]:
%%time
model_regr = LogisticRegression(random_state=12345)
model_regr.fit(features_train, target_train)
predictions = model_regr.predict(features_test)
accuracy = model_regr.score(features_valid, target_test)
print(accuracy)

0.6936236391912908
CPU times: user 49.1 ms, sys: 4.01 ms, total: 53.1 ms
Wall time: 62.8 ms


## (бонус) Проверяем модели на адекватность

In [15]:
target_pred_constant = pd.Series(0, range(0,643))
print(accuracy_score(target_test, target_pred_constant))

0.7060653188180405


Если просто заполнять всю выборку нулевыми значениями, то accuracy уже 70%, так как тарифом Смарт пользуется намного больше людей

In [16]:
from sklearn.metrics import confusion_matrix

display('tree',confusion_matrix(target_valid, predictions_valid_tree))
display('forest',confusion_matrix(target_valid, predictions_valid_forest))
display('regr',confusion_matrix(target_valid, predictions_valid_regr))


'tree'

array([[358,  82],
       [ 83, 120]])

'forest'

array([[390,  50],
       [ 87, 116]])

'regr'

array([[432,   8],
       [195,   8]])

Именно поэтому модель в случае с  1 модели угадывают приблизительно 50/50, а в случае с 0 ошибок уже меньше приблизительно 3:1

Но это также плохой показатель

In [17]:
from sklearn.metrics import precision_score 

print('Точность tree',precision_score(target_valid, predictions_valid_tree))
print('Точность forest',precision_score(target_valid, predictions_valid_forest))
print('Точность regr',precision_score(target_valid, predictions_valid_regr))


Точность tree 0.594059405940594
Точность forest 0.6987951807228916
Точность regr 0.5


In [18]:
from sklearn.metrics import f1_score

print('f1_score tree',f1_score(target_valid, predictions_valid_tree))
print('f1_score forest',f1_score(target_valid, predictions_valid_forest))
print('f1_score regr',f1_score(target_valid, predictions_valid_regr))


f1_score tree 0.5925925925925926
f1_score forest 0.6287262872628726
f1_score regr 0.07305936073059362


### Вывод

Модели неадекватны, но как это исправить знаний ещё не имею

## Вывод

Модель с максимально большим значением accuracy на тестовой выборке оказалась у случайного леса - 0.8055987558320373

Но также она выполняется дольше всех - 211 ms

Наиболее неудачной в реализации оказалась модель логистической регрессии - она уступает в скорости дереву решений и имеет наименьший accuracy

При выборе модели необходимо смотреть на время её выполнения и на accuracy, поэтому наиболее правильным будет выбрать в качестве эталонной модели: дерево решений - accuracy = 0.7822706065318819, но скорость гараздо выше чем у случайного леса = 21.9 ms, в 10 раз выше



## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
